# Sprachmodelle II - Neuronale Netze

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = "{:,.4f}".format

Wir ladene einen ersten Text aus einer Datei und geben ihn aus:

In [ ]:
with open('txt/zauberlehrling.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
print(text)

### Einstieg & Recap

Wieder machen wir den Computer bekannt mit den allen Texten, die er beherrschen soll. Dazu erzeugen wir einen Tokenizer und füttern ihn mit dem Gedicht 'Der Zauberlehrling'. Der Tokenizer zerlegt das Gedicht in Token und merkt sich, welche Token vorkamen.

In [ ]:
from fws.tokenizer import FWSTokenizer
tokenizer = FWSTokenizer(text)
toks = tokenizer.tokenize(text)
df = pd.DataFrame(
    data={'token':tokenizer.vocab_str(), 'output':tokenizer.decode_list(tokenizer.vocab_int())}, 
    index=tokenizer.vocab_int())
df.head(10)

In [ ]:
print(f"Anzahl der Token: {len(toks)}")
print(f"Anzahl der unterschiedlichen Token: {tokenizer.vsize}")

#### Aufgabe 1

##### 0,5 + 0,5 Punkte

- F: Was ist ein Token?
- A:
- F: Warum ist die Anzahl der unterschiedlichen Token kleiner als die Anzahl der Token?
- A:

### Text zu Zahlen in einer Tabelle

Wir erinnern uns, dass Machine Learning Modelle grundsätzlich nur Zahlen ausgeben und auch nur Zahlen als Eingabe akzeptieren. Wir wissen, dass wir mit dem Tokenizer aus Text Zahlen und aus Zahlen Text machen können. Wir schauen uns zunächst an, wie die Eingabedaten für unser erstes Machine Learning Modell aussahen. Auf Basis dieser Daten hat das Modell gelernt, bei welcher Temperatur wie viel Eis verkauft wird.

In [ ]:
df = pd.read_csv('data/eis.csv')
df[["Temperatur", "Eis"]].head(10)

Mit Hilfe des Tokenizers erzeugen wir uns aus dem Text 'Der Zauberlehrling' eine Tabelle mit Trainingsadten:

In [ ]:
xs, ys = [], []

for t1, t2 in zip(toks, toks[1:]):
    ix1 = tokenizer.idx(t1)
    ix2 = tokenizer.idx(t2)
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

xsl = xs.tolist()
ysl = ys.tolist()

data = {"xs": xsl, "ys": ysl, "xs_dec": tokenizer.decode_list(xsl), "ys_dec": tokenizer.decode_list(ysl)}
df = pd.DataFrame.from_dict(data, orient='index').transpose()
df.head()

#### Aufgabe 2

##### 2 + 1 Punkte

Oben sehen Sie die ersten Zeilen der Tabelle mit den Trainingsdaten aus dem Gedicht. 

- F: Finden Sie jeweils einen besseren Namen für die Spalten 'xs' und 'ys'. 
- A:
- F: In der Tabelle Steckt das ganze Gedicht. Wie viele Zeilen hat die Tabelle?
- A:

In [ ]:
W = torch.randn((tokenizer.vsize, tokenizer.vsize), requires_grad=True)
W

In [ ]:
plt.imshow(W.detach().numpy())

In [ ]:
def generate(A, max_tokens, start_token_id):
    gen_idx_seq = [start_token_id]
    while True:
        logits = A[gen_idx_seq[-1]]
        counts = logits.exp()
        probs = counts / counts.sum() 
        idx = torch.multinomial(probs, num_samples=1).item()
        gen_idx_seq.append(idx)
        if len(gen_idx_seq) == max_tokens:
            break
    
    return gen_idx_seq

In [ ]:
tokens = 32
start = tokenizer.idx('<NL>')

id_seq = generate(W, tokens, start)
print(id_seq)
print(tokenizer.decode(id_seq))

#### Aufgabe 3

##### 1 + 2 + 1 + 2 = 6 Punkte

- F: Wie Punkte (Pixel) hat das Bild oben? 
- A:
- F: Führen Sie die Zelle oberhalb mehrfach aus. Was wird hier ausgegeben?
- A:
- F: Warum ändert sich die Ausgabe bei jedem Versuch?
- A:
- F: Was sieht man auf dem Bild oben?
- A:

### Der Lernprozess

Wie in unseren ersten Beispielen mit dem Eisstand, trainieren wir unser Modell jetzt schrittweise indem wir die Parameter schrittweise so verändern, dass der Fehler möglichst schnell kleiner wird.

In [ ]:
def forward(W, xs, ys, learning_rate, v_size):
    xenc = F.one_hot(xs, num_classes=v_size).float()
    logits = xenc @ W 
    counts = logits.exp() 
    probs = counts / counts.sum(1, keepdim=True) #normalize rows
    loss = -probs[torch.arange(xs.shape[0]), ys].log().mean() + learning_rate * (W**2).mean()
    return loss

In [ ]:
def learn(passes, learning_rate, prints=4):
    s = passes / prints
    r = 2
    c = prints + 1
    p = 1
    W = torch.randn((tokenizer.vsize, tokenizer.vsize), requires_grad=True)
    #print(c)
    fig = plt.figure(figsize=(c*4, r*4))
    losses = []
    for k in range(passes):
        loss = forward(W, xs, ys, learning_rate, tokenizer.vsize)
        losses.append(loss.item())
        #backward pass
        W.grad = None
        loss.backward() #will reverse fwd pass ops and update W.grad
        
        #update
        W.data += -1 * W.grad
        
        if(k % s == 0):
            fig.add_subplot(r, c, p).title.set_text('Pass: {passes}, Loss: {ploss:.4f}'.format(passes=k, ploss=loss.item()))
            plt.imshow(W.detach().numpy())
            #fig.add_subplot(r, c, c+p)
            #plt.imshow(W.grad.detach().numpy())
            p += 1
    
    fig.add_subplot(r, c, p).title.set_text('Pass: {passes}, Loss: {ploss:.4f}'.format(passes=k, ploss=loss.item()))
    plt.imshow(W.detach().numpy())
    #fig.add_subplot(r, c, c+p)
    #plt.imshow(W.grad.detach().numpy())
            
    plt.show()
    return W, losses

#### Versuch zu Aufgabe 4

In [ ]:
learning_rate = 0.01
steps = 4096

(W_learned, losses) = learn(steps, learning_rate)

In [ ]:
plt.imshow(W_learned.detach().numpy())

In [ ]:
plt.plot(losses)
plt.show()

#### Aufgabe 4

Führen sie die drei Zellen des Versuch oberhalb mit drei verschiedenen Werten für 'steps' durch: 512, 2048, 4096. Die Beiden Bilder obherhalb zeigen die Gewichte der Neuronen und den durchschnittlichen Fehler des Modells.

##### 4 + 1 + 1 + 2 = 8 Punkte

- F: Beschreiben sie, wie sich die beiden Bilder obenrhalb verändern wenn der Wert für Steps größer wird?
- A:
- F: An welches Bild aus der vorherigen Stunde erinnert Sie das letzte Bild mit 4096 steps?
- A:
- F: Wie gut ist das Modell gegenüber unserem Versuch letzte Stunde?
- A:
- F: Was müssten Sie tun um es noch besser zu machen?
- A:

### Finale

Abschließend können wir aus dem neu trainierten Modell Texte erzeugen

In [ ]:
tokens = 32
start = tokenizer.idx('<NL>')

id_seq = generate(W, tokens, start)
print(id_seq)
print(tokenizer.decode(id_seq))